# Collapse utterances

## Load data using postgres

In [1]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

# Define a database name
dbname = 'phraze_db'
username = 'elahi' # change this to your username

## 'engine' is a connection to a database
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database = dbname, user = username)
con

/Users/elahi/anaconda/envs/py27/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


postgres://elahi@localhost/phraze_db


<connection object at 0x105f06ad0; dsn: 'user=elahi dbname=phraze_db', closed: 0>

In [51]:
# Load data and select relevant columns
sql_query = """SELECT * FROM phraze_data_table;"""
df = pd.read_sql_query(sql_query, con)

# Choose columns to keep
columns_to_keep = ['ID', 'Line', 'UtteranceNumber', 'Role', 
                  'Text', 'Thread_number','note', 'note_cat']

df = df[columns_to_keep]
df.shape

(87003, 8)

In [52]:
# Drop duplicates
df2 = df.drop_duplicates()
print df.shape
print df2.shape
df = df.drop_duplicates()

(87003, 8)
(86274, 8)


In [53]:
import pandas as pd
import numpy as np
import re

# To save output
import pickle
from __future__ import division

In [54]:
# Drop Thread_number or Text values that are None or NA
df.dropna(inplace=True, subset = ['Role','Text','Thread_number','note'])
df.shape
df_original = df

In [127]:
# Subset the data for testing
df = df_original[:20]
print df.shape
df.head()

(20, 8)


,ID,Line,UtteranceNumber,Role,Text,Thread_number,note,note_cat
3,HFC01P008_v_1 NM JB NM,4,4,RN,Looks like 110/70.,1,2,Assessment
4,HFC01P008_v_1 NM JB NM,4,5,RN,I just need a quick heart reading. Y,78,1,Physical exam
5,HFC01P008_v_1 NM JB NM,4,6,RN,"Yep, you're right -- 1",2,2,Assessment
6,HFC01P008_v_1 NM JB NM,4,7,RN,112,2,2,Assessment
7,HFC01P008_v_1 NM JB NM,4,8,RN,So how long has that been going on?,2,0,History of present illness


In [128]:
## Function to compare previous cell and return boolean whether the same value
def comp_prev(a):
    return np.concatenate(([False],a[1:] == a[:-1]))

df['Role_match'] = comp_prev(df.Role.values)
df['Note_match'] = comp_prev(df.note.values)

/Users/elahi/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/elahi/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [129]:
df[:5]

,ID,Line,UtteranceNumber,Role,Text,Thread_number,note,note_cat,Role_match,Note_match
3,HFC01P008_v_1 NM JB NM,4,4,RN,Looks like 110/70.,1,2,Assessment,False,False
4,HFC01P008_v_1 NM JB NM,4,5,RN,I just need a quick heart reading. Y,78,1,Physical exam,True,False
5,HFC01P008_v_1 NM JB NM,4,6,RN,"Yep, you're right -- 1",2,2,Assessment,True,False
6,HFC01P008_v_1 NM JB NM,4,7,RN,112,2,2,Assessment,True,True
7,HFC01P008_v_1 NM JB NM,4,8,RN,So how long has that been going on?,2,0,History of present illness,True,False


In [130]:
## Test if all matches are True, then same number = True
## But if all matches are False, then same number = False

# For each row:
# 1. Extract Line_match, Role_match, Thread_match as array
# 2. Use all() to test whether all are True
# 3. Return this boolean as a new column

In [131]:
test_row = df[0:1]
test_array = np.array(test_row[['Role_match', 'Note_match']])[0]
print test_array
test_boolean = all(test_array)
print test_boolean

[False False]
False


In [132]:
a = []
for i in range(1, len(df)+1):
    test_row = df[i-1:i]
    test_array = np.array(test_row[['Role_match', 'Note_match']])[0]
    test_boolean = all(test_array)
    a.append(test_boolean)
#print a
print len(a)

20


In [133]:
print df.shape
df['same_number'] = a

(20, 10)


/Users/elahi/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [134]:
df.head()

,ID,Line,UtteranceNumber,Role,Text,Thread_number,note,note_cat,Role_match,Note_match,same_number
3,HFC01P008_v_1 NM JB NM,4,4,RN,Looks like 110/70.,1,2,Assessment,False,False,False
4,HFC01P008_v_1 NM JB NM,4,5,RN,I just need a quick heart reading. Y,78,1,Physical exam,True,False,False
5,HFC01P008_v_1 NM JB NM,4,6,RN,"Yep, you're right -- 1",2,2,Assessment,True,False,False
6,HFC01P008_v_1 NM JB NM,4,7,RN,112,2,2,Assessment,True,True,True
7,HFC01P008_v_1 NM JB NM,4,8,RN,So how long has that been going on?,2,0,History of present illness,True,False,False


In [135]:
# Assign new number based on boolean
b = [1]
i = 1
same_number = df.same_number[i-1:i].values
print(same_number)
print(same_number == True)

if same_number == True:
    print i
else: 
    print i+1

b

[False]
[False]
2


[1]

In [144]:
b = [10, 21, 3, 1, 0]
b[-1]

0

In [145]:
b = [1]
for i in range(2, len(df)+1):
    same_number = df.same_number[i-1:i].values
    if same_number == True:
        b.append(b[-1])
    else:
        b.append(b[-1]+1)
len(b)

20

In [146]:
print b
print df.same_number.values

[1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 9, 9, 9, 10, 10, 11, 12, 13, 14, 15]
[False False False  True False False False False False False  True  True
  True False  True False False False False False]


In [147]:
df['phrase'] = b

/Users/elahi/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [148]:
df.to_csv("data_output/df_test_phrase_number.csv")

In [149]:
# Prep for the loop
unique_lines = df.phrase.unique()
lines_n = len(unique_lines)
lines_n

15

In [154]:
# Initialize the dataset
i = 0
line_i = unique_lines[0]
dat_new = df[df['phrase'] == line_i]
dat_new

,ID,Line,UtteranceNumber,Role,Text,Thread_number,note,note_cat,Role_match,Note_match,same_number,phrase
3,HFC01P008_v_1 NM JB NM,4,4,RN,Looks like 110/70.,1,2,Assessment,False,False,False,1


In [155]:
lines_n
range(1, lines_n)
for i in range(1, lines_n):
    line_i = unique_lines[i]
    dat_i = df[df['phrase'] == line_i]
    
    a = []
    for j in dat_i.Text:
        a.append(j)
        a2 = ' '.join(a)
    
    dat_i_slice = dat_i[0:1].reset_index()
    dat_i_slice.drop('Text', axis = 1, inplace=True) # Drop text
    dat_i_slice['Text'] = a2
    
    dat_new = dat_new.append(dat_i_slice)

/Users/elahi/anaconda/envs/py27/lib/python2.7/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [157]:
# Export testing dataset for visual checks
dat_new.to_csv("data_output/df_sub_new.csv")
df.to_csv("data_output/df_original.csv")

## Run on entire dataset

In [162]:
print df.shape

(83817, 12)


,ID,Line,UtteranceNumber,Role,Text,Thread_number,note,note_cat,Role_match,Note_match,same_number,phrase
86995,RI12P122_v_1_CD NM,184,444,P,Yes! I,6,3,Plan,False,True,False,50980
86996,RI12P122_v_1_CD NM,184,445,P,I win one.,6,3,Plan,True,True,True,50980
86997,RI12P122_v_1_CD NM,185,446,D,"All right, so let's see here. All right, c",33,6,Logistics,False,False,False,50981
86998,RI12P122_v_1_CD NM,185,447,D,"come on, I'll meet you outside.",33,6,Logistics,True,True,True,50981
86999,RI12P122_v_1_CD NM,186,448,P,Okay.,33,6,Logistics,False,True,False,50982


In [160]:
df = df_original

## Function to compare previous cell and return boolean whether the same value
def comp_prev(a):
    return np.concatenate(([False],a[1:] == a[:-1]))
df['Role_match'] = comp_prev(df.Role.values)
df['Note_match'] = comp_prev(df.note.values)

## Does the speaker role and note of the current row match the previous row?
## (i.e., should this be the same phrase number? True/False)
a = []
for i in range(1, len(df)+1):
    test_row = df[i-1:i]
    test_array = np.array(test_row[['Role_match', 'Note_match']])[0]
    test_boolean = all(test_array)
    a.append(test_boolean)
df['same_number'] = a

## Assign phrase numbers, starting with 1. 
## If same_number == True, use previous number; otherwise use previous number + 1
b = [1]
for i in range(2, len(df)+1):
    same_number = df.same_number[i-1:i].values
    if same_number == True:
        b.append(b[-1])
    else:
        b.append(b[-1]+1)        
df['phrase'] = b

KeyboardInterrupt: 

In [166]:
print df.tail()

                       ID  Line  UtteranceNumber Role  \
86995  RI12P122_v_1_CD NM   184              444    P   
86996  RI12P122_v_1_CD NM   184              445    P   
86997  RI12P122_v_1_CD NM   185              446    D   
86998  RI12P122_v_1_CD NM   185              447    D   
86999  RI12P122_v_1_CD NM   186              448    P   

                                              Text Thread_number  note  \
86995                                       Yes! I             6     3   
86996                                   I win one.             6     3   
86997  All right, so let's see here.  All right, c            33     6   
86998              come on, I'll meet you outside.            33     6   
86999                                        Okay.            33     6   

        note_cat  Role_match  Note_match  same_number  phrase  
86995       Plan       False        True        False   50980  
86996       Plan        True        True         True   50980  
86997  Logistics    

In [167]:
## Concatenate the Text, by phrase number 
# Prep for the loop
unique_lines = df.phrase.unique()
lines_n = len(unique_lines)

# Initialize the dataset
i = 0
line_i = unique_lines[0]
dat_new = df[df['phrase'] == line_i]

# Run the loop
for i in range(1, lines_n):
    line_i = unique_lines[i]
    dat_i = df[df['phrase'] == line_i]
    
    a = []
    for j in dat_i.Text:
        a.append(j)
        a2 = ' '.join(a)
    
    dat_i_slice = dat_i[0:1].reset_index()
    dat_i_slice.drop('Text', axis = 1, inplace=True) # Drop text
    dat_i_slice['Text'] = a2
    
    dat_new = dat_new.append(dat_i_slice)

In [168]:
# started at 2:07pm
# finished at 2:59pm
dat_new.shape

(50982, 13)

In [169]:
dat_new.to_csv("data_output/phraze_data_collapsed.csv")